In [2]:
import snntorch as snn
import torch
import torch.nn as nn
from snntorch import spikeplot as splt
from torch.utils.data import DataLoader
# from torchvision import datasets, transforms
import numpy as np
import itertools
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
import network
import torch.optim as optim
import os
from PIL import Image
import torchvision.transforms.functional as TF
import cv2
import pandas as pd
import time
import data as dt


# os.environ["CUDA_LAUNCH_BLOCKING"] = "1" # for debugging on GPU
%load_ext autoreload
%autoreload 2

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
dataset_path = 'dataset_python_sim'
accumulation_time = 10 # ms
quantization = 8
batch_size = 4
dataset_type = "all"
labels = ['x_cam', 'y_cam', 'R_cam', 'in_fov']
trainset = dt.Tracking3DVideoDataset(dataset_path, accumulation_time, quantization = quantization, split='train', dataset_type=dataset_type, labels=labels)
testset = dt.Tracking3DVideoDataset(dataset_path, accumulation_time, quantization = quantization, split='test', dataset_type=dataset_type, labels=labels)
validationset = dt.Tracking3DVideoDataset(dataset_path, accumulation_time, quantization = quantization, split='val', dataset_type=dataset_type, labels=labels)
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=0, collate_fn=trainset.collate_fn)
testloader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=0, collate_fn=testset.collate_fn)
valid_loader = DataLoader(validationset, batch_size=batch_size, shuffle=False, num_workers=0, collate_fn=validationset.collate_fn)

Loaded 21531 rows from dataset_python_sim\positions_10ms.csv
Found 156 sequences in dataset_python_sim\positions_10ms.csv
Shape of the images: torch.Size([2, 90, 160])
Label shape: None
Split sizes: train=109, val=23, test=24, total=156
Final dataset split='train' size: 109
Loaded 21531 rows from dataset_python_sim\positions_10ms.csv
Found 156 sequences in dataset_python_sim\positions_10ms.csv
Shape of the images: torch.Size([2, 90, 160])
Label shape: None
Split sizes: train=109, val=23, test=24, total=156
Final dataset split='test' size: 24
Loaded 21531 rows from dataset_python_sim\positions_10ms.csv
Found 156 sequences in dataset_python_sim\positions_10ms.csv
Shape of the images: torch.Size([2, 90, 160])
Label shape: None
Split sizes: train=109, val=23, test=24, total=156
Final dataset split='val' size: 23


In [7]:
model = network.load_model("models/VideoClassification_w_confidence_finetuned_lower_lr_q8_5ts_10e.pt", network.SCNNVideoClassification, trainset=trainset, device=device)
model.evaluate(valid_loader, device, num_steps = 5, print_results=True)

Loading model with weighted average: False
Flattened feature size: 14080
Number of x_cam bins: 161
Number of y_cam bins: 91
Number of R_cam bins: 101
Number of in_fov bins: 1
Evaluating video classification tracker
Average Error for x_cam: 7.2228 pixels
Average Error for y_cam: 5.9939 pixels
Average Error for R_cam: 0.6192 pixels
Average Error for in_fov: 0.0823 pixels


array([7.2228027 , 5.99385372, 0.61923786, 0.08226998])